In [1]:
import talib as ta
import numpy as np 
import requests 
import pandas as pd 
import matplotlib.pyplot as plt 
import matplotlib.gridspec as gridspec 
from datetime import datetime,timedelta 
from mplfinance.original_flavor import candlestick_ohlc

In [2]:
tf = '1h'
symbol = 'DOGEUSDT' 

In [3]:
binance_s_url  = 'https://api.binance.com/api/v3/'  

In [4]:
def indicator__macd(data, fastperiod=12, slowperiod=26, signalperiod=9): #MACD
    close  = np.array(data[:,4], dtype=float)
    return ta.MACD(close, fastperiod, slowperiod, signalperiod)

def tostr(val, precision):
    return f'{float(val):.{precision}f}' 

def binance__s_getCandles(interval, symbol, limit):
    return np.array(requests.get(binance_s_url+'klines?symbol='+symbol+'&interval='+interval+'&limit='+str(limit)).json())

In [5]:
data = binance__s_getCandles(tf, symbol, 2000)[:-1]   

In [6]:
macd, signal, hist = indicator__macd (data)

In [7]:
def indicator__plot_macd( data, sym, tf, indicators, pp):
    
    histo      = indicators[2][~np.isnan(indicators[0])]
    macd       = indicators[0][-len(histo):]
    signal     = indicators[1][-len(histo):]
    
    indicators = np.vstack([macd, signal, histo])
        
    data_trim  = data[-len(indicators[2]):]
    x          = np.array(range(-len(data_trim),0))
    ohlc_data  = data_trim[:, [1, 2, 3, 4]].astype(float)
    high       = np.array(data_trim[:,2], dtype=float)
    low        = np.array(data_trim[:,3], dtype=float)
    
    ohlc    = np.hstack((x.reshape(-1, 1), ohlc_data))
    fig     = plt.figure(figsize=(8, 6))
    gs      = gridspec.GridSpec(2, 1, height_ratios=[8, 2], hspace=0)
    ax      = plt.subplot(gs[0])
    ax_indi = plt.subplot(gs[1])
    
    # Plot the candlestick
    candlestick_ohlc(ax, ohlc, width=0.5, colorup='#16937E', colordown='#EC3847', alpha=1)
    ax.set_facecolor('#141924')
    ax.grid(True, linestyle='--', color='#373B44', linewidth=1)
    ax.set_xticks([])
    ax.yaxis.tick_right()
    ax.tick_params(axis='y', labelcolor='#B2B4BF', labelsize=8)
    ax.set_xlim(-len(data_trim)-2, 2)
    ax.set_ylim(0.98*np.min(low), 1.02*np.max(high))

    ax_indi.set_facecolor('#141924')
    ax_indi.grid(True, linestyle='--', color='#373B44', linewidth=0.5)
    ax_indi.set_xticks([])
    ax_indi.yaxis.tick_right()
    ax_indi.tick_params(axis='y', labelcolor='#B2B4BF', labelsize=8)

    # Plot the indicator
    ax_indi.plot(x, macd, color='#2962FF', linewidth=0.75)
    ax_indi.plot(x, signal, color='#FF6D00', linewidth=0.75)
    colors = np.full_like(histo, '#000000', dtype='<U7')
    diffs  = np.concatenate((np.array([0]), np.diff(histo)))
    colors[(histo >= 0) & (diffs >= 0)] = '#26A69A'
    colors[(histo >= 0) & (diffs <  0)] = '#B2DFDB'
    colors[(histo <= 0) & (diffs <= 0)] = '#FF5252'
    colors[(histo <= 0) & (diffs >  0)] = '#FFCDD2'
    ax_indi.bar(x, histo, color=colors)
    
    i = 0
    xlim = -len(indicators[0])
    y_max = -100000
    y_min = 100000
    for indicator in indicators:
        y_max = max(y_max, 1.01*np.max(indicator))
        y_min = min(y_min, 1.01*np.min(indicator)) if (np.min(indicator) < 0) else min(y_min, 0.99*np.min(indicator))
        i += 1
    ax_indi.set_xlim(xlim-2, 2)
    ax_indi.set_ylim(y_min, y_max)
        
    formatted_time = datetime.now().strftime("%b %d, %Y %H:%M") + "+5:30"     
    title = f"{formatted_time}\n\n{sym}, {tf}, • O {tostr(data[-1][1],pp)} • H {tostr(data[-1][2],pp)} • L {tostr(data[-1][3],pp)} • C {tostr(data[-1][4],pp)}\n"

    ax.text(0, 1, title, color='white', alpha=0.5, fontsize=9, ha='left', va='top', rotation=0, zorder=1, transform=ax.transAxes)
    plt.subplots_adjust(hspace=0, wspace=0)
    plt.margins(0, 0)
    plt.tight_layout()
    filename = f"{sym}_{tf}_VWMACD_{datetime.now().strftime('%d%b_%H%M')}.png"
    plt.savefig(filename, facecolor='#141C24', dpi=200)
    plt.clf()
    return filename

In [8]:
indicator__plot_macd( data, symbol, tf, indicators=np.vstack([macd, signal, hist]), pp=4)

'DOGEUSDT_1h_VWMACD_03Aug_2049.png'

<Figure size 800x600 with 0 Axes>

In [9]:
open_ = np.array(data[:,1], dtype=float)
net_qnty = 0 
balance = 1000
price_ = 100


orders = {}

for i in range(1,len(macd)):
    buy  = hist[i-1]<0 and hist[i]>0 and macd[i]<0 and signal[i]<0
    sell = hist[i-1]>0 and hist[i]<0 and macd[i]>0 and signal[i]>0

    if (buy):
        qnty = price_/open_[i]
        if (balance < price_):
            continue
        net_qnty+=qnty
        balance -= price_
        print (f"BUY {qnty:.4f} at {i} : Price = {open_[i]} \u2022 Net : {net_qnty:.4f} \u2022 Balance {balance}")
        orders[i] = [open_[i],net_qnty,'BUY','0',round(balance,4)]

    if (sell and net_qnty > 0):
        qnty = price_/open_[i]
        net_qnty = net_qnty - qnty if (net_qnty > qnty) else 0
        balance += price_
        profit = net_qnty*open_[i]    

        print (f"SELL {qnty:.4f} at {i} : Price = {open_[i]} \u2022 Net : {net_qnty:.4f} \u2022 Balance {balance} \u2022 Profit : {profit:.4f}")
        orders[i] = [open_[i],net_qnty,'SELL',round(profit,4),round(balance,4)]    

BUY 861.1039 at 45 : Price = 0.11613 • Net : 861.1039 • Balance 900
SELL 792.6443 at 72 : Price = 0.12616 • Net : 68.4597 • Balance 1000 • Profit : 8.6369
BUY 821.4227 at 105 : Price = 0.12174 • Net : 889.8824 • Balance 900
SELL 797.7026 at 126 : Price = 0.12536 • Net : 92.1798 • Balance 1000 • Profit : 11.5557
SELL 789.0169 at 133 : Price = 0.12674 • Net : 0.0000 • Balance 1100 • Profit : 0.0000
BUY 806.6468 at 152 : Price = 0.12397 • Net : 806.6468 • Balance 1000
BUY 822.9775 at 175 : Price = 0.12151 • Net : 1629.6243 • Balance 900
SELL 796.3051 at 202 : Price = 0.12558 • Net : 833.3192 • Balance 1000 • Profit : 104.6482
BUY 807.6892 at 227 : Price = 0.12381 • Net : 1641.0084 • Balance 900
BUY 810.5041 at 234 : Price = 0.12338 • Net : 2451.5125 • Balance 800
BUY 845.1657 at 263 : Price = 0.11832 • Net : 3296.6781 • Balance 700
BUY 928.5051 at 284 : Price = 0.1077 • Net : 4225.1833 • Balance 600
BUY 1027.8549 at 298 : Price = 0.09729 • Net : 5253.0381 • Balance 500
SELL 899.1189 at 33

In [12]:
data_df = [{'Price': price, 'Net_Qnty': net_qnty, 'Profit' : profit, 'Balance':balance, 'Order' : order} for price, 
           net_qnty, order, profit, balance in orders.values()] 

In [13]:
orders_df = pd.DataFrame(data_df) 
orders_df  

,Price,Net_Qnty,Profit,Balance,Order
0,0.11613,861.103935,0,900,BUY
1,0.12616,68.459674,8.6369,1000,SELL
2,0.12174,889.882378,0,900,BUY
3,0.12536,92.179762,11.5557,1000,SELL
4,0.12674,0.000000,0.0,1100,SELL
5,0.12397,806.646769,0,1000,BUY
6,0.12151,1629.624302,0,900,BUY
7,0.12558,833.319158,104.6482,1000,SELL
8,0.12381,1641.008359,0,900,BUY
9,0.12338,2451.512493,0,800,BUY
